CAPM: Capital Asset Pricing Model

+ Descreve as relações entre o retorno esperado e o risco, comparando o portfólio com o mercado.

Fórmula:

--> Ri = Rf + Bi * (Rm - Rf)

Rm: retorno esperado do portfólio.

Rf: risk-free.

Beta: comparação entre a carteira e o mercado.

  - Se =1: fortemente correlacionado com o mercado.

  - Se <1: (defensivo) menos volátil que o mercado.

  - Se >1: (agressivo) mais volátil que o mercado.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
stocks = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA', 'NQ=F']
main = pd.DataFrame()

for stock in stocks:
  main[stock] = yf.download(stock, start = '2024-01-01', end = '2024-07-12')['Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
main = main.reset_index(drop = True)

In [ ]:
normalized = main.copy()
i = 1 #removing the 'index' column
for i in main.columns:
  normalized[i] = normalized[i] / normalized[i][0]

normalized

,AAPL,MSFT,GOOG,AMZN,NVDA,NQ=F
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.992512,0.999272,1.005732,0.990262,0.987564,0.989115
2,0.979907,0.992100,0.989109,0.964250,0.996471,0.983553
3,0.975975,0.991587,0.984451,0.968719,1.019287,0.984465
4,0.999569,1.010300,1.006950,0.994464,1.084807,1.005009
...,...,...,...,...,...,...
127,1.219242,1.260711,1.375466,1.333956,2.612315,1.233298
128,1.227214,1.257152,1.364861,1.329220,2.661518,1.235631
129,1.231847,1.239087,1.364574,1.329554,2.727537,1.236752
130,1.255010,1.257179,1.380482,1.332555,2.800822,1.249985


In [ ]:
return_rate = (normalized / normalized.shift(1)) - 1

In [ ]:
return_rate.fillna(0, inplace = True)
display(return_rate)

,AAPL,MSFT,GOOG,AMZN,NVDA,NQ=F
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.007488,-0.000728,0.005732,-0.009738,-0.012436,-0.010885
2,-0.012700,-0.007178,-0.016529,-0.026268,0.009018,-0.005623
3,-0.004013,-0.000516,-0.004709,0.004634,0.022897,0.000927
4,0.024175,0.018872,0.022855,0.026577,0.064281,0.020868
...,...,...,...,...,...,...
127,0.021620,0.014736,0.024388,0.012197,-0.019099,0.010252
128,0.006539,-0.002823,-0.007710,-0.003550,0.018835,0.001891
129,0.003775,-0.014370,-0.000210,0.000251,0.024805,0.000908
130,0.018804,0.014602,0.011657,0.002257,0.026869,0.010700


In [ ]:
return_rate.mean() * 132

AAPL    0.219003
MSFT    0.213513
GOOG    0.314139
AMZN    0.279091
NVDA    1.044962
NQ=F    0.207163
dtype: float64

In [ ]:
beta, alpha = np.polyfit(x = return_rate['NQ=F'],
                         y = return_rate['NVDA'],
                         deg = 1)

print(f'beta: {beta}')
print(f'alpha (%): {alpha * 100}')

beta: 2.2431961587351155
alpha (%): 0.439587049920169


CONCLUSÃO:

Parâmetro beta: Compara a volatidade (indicou que o mercado Y (NVIDIA) é 124,2% mais volátil que o mercado X (NASDAQ))

Parâmetro alpha: Indica excesso de retorno (mercado Y é 43,3% mais lucrativo que o mercado X)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(return_rate,
                 x = 'NQ=F',
                 y = 'NVDA')
fig.add_scatter(x = return_rate['NQ=F'],
                y = beta * return_rate['NQ=F'] + alpha)
fig.show()

In [ ]:
beta * 0.01995883 + alpha # APLICAÇÃO MANUAL PARA TESTAR NA LINHA

0.049167441288048876

Calculo do BETA por meio da covariância e variância:

Bi = cov (x,y) / var (x)

In [ ]:
matrix_cov = return_rate.drop(columns = ['AMZN', 'AAPL', 'GOOG', 'MSFT']).cov() * 133
display(matrix_cov)

cov_xy = matrix_cov.iloc[1,0]
print(f'cov: {cov_xy}')

var_x = return_rate['NQ=F'].var() * 133
print(f'var x: {var_x}')

b = cov_xy / var_x
print(f'beta: {b}')

,NVDA,NQ=F
NVDA,0.142176,0.027527
NQ=F,0.027527,0.012271


cov: 0.027526798083165566
var x: 0.01227123984497515
beta: 2.243196158735117


In [ ]:
rf = np.array([10.75])
rf = rf.mean() / 100
rf

0.1075

In [ ]:
rm_list = []

for i in return_rate.columns[0:-1]:
  rm_list.append(return_rate[i].mean() * 132)

In [ ]:
rm_list

[0.21900327480665915,
 0.21351332938580214,
 0.3141392950161933,
 0.27909138206399187,
 1.0449624859470212]

In [ ]:
betas =  []
alphas = []

for i in return_rate.columns[0:-1]:
  beta, alpha = np.polyfit(x = return_rate['NQ=F'],
                           y = return_rate[i],
                           deg = 1)
  betas.append(beta)
  alphas.append(alpha)

In [ ]:
i = 0
for name in return_rate.columns[0:-1]:
  print(f'For {name}:')
  print(f'Betas: {betas[i]}')
  print(f'Alphas: {alphas[i] * 100}')
  i+=1
  print('-=' * 15)



For AAPL:
Betas: 0.8318264717207757
Alphas: 0.03536324508281042
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
For MSFT:
Betas: 0.9982470062322902
Alphas: 0.005085860683304312
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
For GOOG:
Betas: 0.9850521035592763
Alphas: 0.08338848433390127
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
For AMZN:
Betas: 1.04250503173888
Alphas: 0.047820270339391735
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
For NVDA:
Betas: 2.2431961587351155
Alphas: 0.439587049920169
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [ ]:
rm = return_rate['NQ=F'].mean() * 132
rm

0.2071631489929242

In [ ]:
CAPMs = []

for i in range(0, len(betas)):
  capm = rf + betas[i] * (rm - rf)
  CAPMs.append(capm)

print(CAPMs)

[0.19040244558736613, 0.2069884401138693, 0.20567339456282158, 0.21139933430406516, 0.3310639929883731]


In [ ]:
pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
CAPM_portfolio = np.sum(CAPMs * pesos) * 100
CAPM_portfolio

22.910552151129906

O retorno é 22% maior que o risco a ser corrido